In [73]:
import neo4j
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [99]:
load_dotenv()

STORAGE_ACCOUNT_NAME = os.getenv('STORAGE_ACCOUNT_NAME')
STORAGE_ACCOUNT_KEY = os.getenv('STORAGE_ACCOUNT_KEY')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

builder = SparkSession.builder\
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.4,\
io.delta:delta-core_2.12:2.4.0,\
io.delta:delta-storage:2.4.0")\
        .config(f"fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net", f"{STORAGE_ACCOUNT_KEY}")\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = builder.getOrCreate()

## Product Nodes

In [100]:
root = 'abfss://default@stdatalakeakita.dfs.core.windows.net/synapse/workspaces/syn-synfactoreddatathon01-dev'

df = spark.read\
.format('parquet')\
.load(f'{root}/neo4j/products/**')\
.withColumnRenamed('2nd_category', 'second_category')\
.withColumnRenamed('3rd_category', 'third_category')\
.withColumnRenamed('4th_category', 'fourth_category')\

df.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------+
|      asin|               title|               brand|      main_category|     second_category|      third_category|     fourth_category|  rank|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------+
|B0002WSK8A|Lauren LA100C 39-...|              Lauren|Musical Instruments|             Guitars|Classical &amp; N...|                null|124391|
|B0002FOHTI|Jones JR101MS Obo...|               Jones|Musical Instruments|Instrument Access...|Wind & Woodwind A...|               Reeds|  2949|
|B0002F6K4S|S Series Splash C...|                    |Musical Instruments|  Drums & Percussion|Drum Sets & Set C...|             Cymbals|524988|
|B00061UJBI|Pro Co 3' IPBQ2Q-...|              Pro Co|Musical Instruments|Live Sound &amp; ...|Stage &amp; Studi...|   Instrument 

In [101]:
df.toPandas().to_csv('temp/products.csv', header=True, index=False)

In [79]:
from neo4j import GraphDatabase
import json
driver = GraphDatabase.driver('neo4j+s://5e7957dd.databases.neo4j.io:7687', auth=('neo4j', NEO4J_PASSWORD))


In [84]:
driver.execute_query("""
    LOAD CSV WITH HEADERS FROM 'file:///products.csv' as row
    WITH row WHERE row.asin IS NOT NULL
    MERGE (p:Products {
        id: row.asin,
        asin: row.asin, 
        title: row.title, 
        brand: COALESCE(row.brand, 'not found'),
        main_category: COALESCE(row.main_category, 'not found'),
        second_category: COALESCE(row.second_category, 'not found'), 
        third_category: COALESCE(row.third_category, 'not found'), 
        fourth_category: COALESCE(row.fourth_category, 'not found'),
        rank: COALESCE(row.rank, 'not found')})
    """
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000001A08B72F460>, keys=[])

## Customer Nodes

In [102]:
root = 'abfss://default@stdatalakeakita.dfs.core.windows.net/synapse/workspaces/syn-synfactoreddatathon01-dev'

df = spark.read\
.format('parquet')\
.load(f'{root}/neo4j/customers/**')

df.show()

+--------------+
|   reviewer_id|
+--------------+
|A3BOAYLC2OP2NF|
|A1UW013BFJUTXJ|
|A39MX8JDGSS0ZG|
|A2N3PO11MVLCME|
|A2K99VYA7L0WAU|
|A3C7PMCLAJ3SD2|
|A30KJ99QX32SG1|
|A1BV4T1Z5759VP|
|A36501MWDY5ANK|
| AL1DO54IYBNYT|
|A30K1UAL77VB1A|
|A1IR0HICH5PVC6|
|A1YO1LFYOITMV9|
|A34VHT94QBTMR7|
|A1XSRMPA72NP2T|
|A36IVDWLLJDHNK|
|A3D6IZ9DP2UXYX|
|A12WAZTOG92ROR|
|A2AX3KL9LDO00X|
|A3OH9JQTW5R3R2|
+--------------+
only showing top 20 rows



In [103]:
df.toPandas().to_csv('temp/customers.csv', header=True, index=False)

In [86]:
load_dotenv()
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver('neo4j://localhost:7687', auth=('neo4j', NEO4J_PASSWORD))

driver.execute_query("""
    LOAD CSV WITH HEADERS FROM 'file:///customers.csv' as row
    WITH row WHERE row.reviewer_id IS NOT NULL
    MERGE (c:Customers {reviewer_id: row.reviewer_id})
    """
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000001A08B3BCE50>, keys=[])

## Review Relatioships

In [104]:
root = 'abfss://default@stdatalakeakita.dfs.core.windows.net/synapse/workspaces/syn-synfactoreddatathon01-dev'

df = spark.read\
.format('parquet')\
.load(f'{root}/neo4j/reviews/**')\
.withColumn('review_timestamp', F.col('review_timestamp').cast('string'))
df.show()

+--------------+-------+--------------------+--------------------+----------+--------+
|   reviewer_id|overall|             summary|    review_timestamp|      asin|    type|
+--------------+-------+--------------------+--------------------+----------+--------+
|A3TBJ93ULYA7HX|    5.0|           very good|2023-07-28 18:48:...|B0002F751U|REVIEWED|
| AS3JHENSQ43RV|    5.0|        none better.|2023-07-29 23:14:...|B0002GZSSG|REVIEWED|
|A2ARISNBYH1NC4|    5.0|The only strings ...|2023-07-28 20:41:...|B0002FO3LU|REVIEWED|
|A15MKE6IFGO6ZH|    5.0|The is the humbuc...|2023-07-28 21:39:...|B0002GLAEW|REVIEWED|
|A1W98JOPERVYAA|    5.0|          Five Stars|2023-07-30 00:34:...|B0002I8Y6W|REVIEWED|
|A2YKBFCDRJIR26|    5.0|Difficult to play...|2023-07-27 21:10:...|B0002F4WCU|REVIEWED|
|A1PD4ZVU9KRRME|    5.0|KEYBOARD CHORD CO...|2023-08-02 00:15:...|B00078HZJE|REVIEWED|
|A2A8CQJYR4YGMU|    5.0|          Five Stars|2023-08-02 22:55:...|B0009OW988|REVIEWED|
| ASLA66R1BK8AG|    5.0|Love it, love it,..

In [105]:
df.toPandas().to_csv('temp/review.csv', header=True, index=False)

In [98]:
load_dotenv()
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver('neo4j://localhost:7687', auth=('neo4j', NEO4J_PASSWORD))

driver.execute_query("""
    LOAD CSV WITH HEADERS FROM 'file:///review.csv' as row
    WITH row WHERE row.reviewer_id IS NOT NULL
    MATCH 
        (p:Products {asin: row.asin}),
        (c:Customers {reviewer_id: row.reviewer_id})
    CREATE (c)-[:REVIEWED{overall: row.overall, summary: row.summary, review_timestamp:row.review_timestamp}]->(p);
    """
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x000001A09D1FEE80>, keys=[])

## Second Category Nodes

In [106]:
root = 'abfss://default@stdatalakeakita.dfs.core.windows.net/synapse/workspaces/syn-synfactoreddatathon01-dev'

df = spark.read\
.format('parquet')\
.load(f'{root}/neo4j/second_category/**')

df.show()

+--------------------+
|     second_category|
+--------------------+
|Wind &amp; Woodwi...|
|        Bass Guitars|
|Instrument Access...|
|    Electronic Music|
|Studio Recording ...|
|Microphones & Acc...|
|Amplifiers & Effects|
|  Drums & Percussion|
|    Band & Orchestra|
|Amplifiers &amp; ...|
|             Guitars|
|  Live Sound & Stage|
|Wind & Woodwind I...|
|Drums &amp; Percu...|
|            Ukuleles|
|Microphones &amp;...|
|Live Sound &amp; ...|
|Stringed Instruments|
|Band &amp; Orchestra|
|    Keyboards & MIDI|
+--------------------+
only showing top 20 rows



In [107]:
df.toPandas().to_csv('temp/second_category.csv', header=True, index=False)

## Brand Nodes

In [108]:
root = 'abfss://default@stdatalakeakita.dfs.core.windows.net/synapse/workspaces/syn-synfactoreddatathon01-dev'

df = spark.read\
.format('parquet')\
.load(f'{root}/neo4j/brand/**')

df.show()

+--------------------+
|               brand|
+--------------------+
|                Nady|
|       Propellerhead|
|           Vic Firth|
|            DigiTech|
|by\n    \n    Pri...|
|                Dava|
|    Latin Percussion|
|       Austin Bazaar|
|           Gibraltar|
|    Meinl Percussion|
|         Blaze Audio|
| Lark in the Morning|
|                 AMG|
|            Hamilton|
|          Ernie Ball|
|            Generic.|
| D'Addario Woodwinds|
|         Slide-O-Mix|
|               WUHAN|
|       Clayton Picks|
+--------------------+
only showing top 20 rows



In [109]:
df.toPandas().to_csv('temp/brand.csv', header=True, index=False)